In [1]:
from kipoi_veff import score_variants
import pandas as pd
from kipoi_cadd.utils import load_pickle, decompose_variant_string, generate_intervals_file
from kipoi import get_model

Using TensorFlow backend.


In [2]:
training_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
sample_intervals_file = "/data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/example_files/intervals_file"
sample_fasta_file = "/data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/example_files/fasta_file"
kipoi_features_dir = "/s/project/kipoi-cadd/data/processed/v1.3/kipoi_features/"
intervals_file = kipoi_features_dir + "intervals_10k.tsv"
fasta_file = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa"
shuff_10k_file = training_dir + "ids_10k.pkl"

In [14]:
# To trigger the download of sample intervals and sample fasta files, we have to do this:
model = get_model('DeepSEA/variantEffects')
pred = model.pipeline.predict_example()

Using downloaded and verified file: /data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


0.00B [00:00, ?B/s]

8.19kB [00:00, 20.0kB/s]                 
303kB [00:00, 2.81MB/s]
  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  3.46it/s]


In [13]:
sample_intervals = pd.read_csv(sample_intervals_file, sep='\t', header=None, names=['chr', 'start', 'end'])
"""
with open(sample_fasta_file, "r") as f:
    for line in f:
        print(line)
        break
"""
sample_intervals.head()

,chr,start,end
0,chr22,136018,136069
1,chr22,136351,136402
2,chr22,137749,137800
3,chr22,134503,134554
4,chr22,139010,139061


## Generate intervals file for the variants of our interest

In [32]:
var_ids = load_pickle(shuff_10k_file)
generate_intervals_file(var_ids, intervals_file, use_chr_word=False, header=None)

In [ ]:
model = get_model('DeepSEA/variantEffects')
dl_kwargs = {'intervals_file': intervals_file, 'fasta_file': fasta_file}

In [7]:
# Get the dataloader and instantiate it
dl = model.default_dataloader(**{'intervals_file': intervals_file, 'fasta_file': fasta_file, 'num_chr_fasta': True})
it = dl.batch_iter(batch_size=64)
preds = None
for batch in it:
    if preds is None:
        preds = model.predict_on_batch(batch['inputs'])
    else:
        preds = np.concatenate([preds, model.predict_on_batch(batch['inputs'])], axis=0)
preds.shape

array([[3.98645550e-02, 5.88591094e-04, 8.94931033e-02, ...,
        6.55408902e-03, 6.56132549e-02, 7.31519517e-03],
       [1.02449907e-03, 2.00078313e-04, 1.07573438e-03, ...,
        2.59452909e-02, 7.92154744e-02, 1.52810579e-04],
       [6.32132776e-03, 1.06681315e-02, 7.64664495e-03, ...,
        7.73444101e-02, 3.92744362e-01, 1.42737338e-02],
       ...,
       [1.20406123e-02, 1.59813259e-02, 1.28443195e-02, ...,
        4.10337657e-01, 5.05032957e-01, 1.27899591e-02],
       [4.51430771e-03, 3.04261479e-03, 5.15099289e-03, ...,
        7.72295427e-03, 8.25386643e-02, 5.23434347e-03],
       [3.54196847e-04, 3.60575541e-05, 2.24278177e-04, ...,
        6.14551734e-03, 4.21312340e-02, 1.65928795e-04]], dtype=float32)

In [ ]:
# Signature
score_variants(model,
               dl_args,
               input_vcf,
               output_vcf,
               scores=["logit_ref", "logit_alt", "ref", "alt", "logit", "diff"],
               score_kwargs=None,
               num_workers=0,
               batch_size=32,
               source='kipoi',
               seq_length=None,
               std_var_id=False,
               restriction_bed=None,
               return_predictions=False,
               model_outputs = None)